In [1]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer,BertForSequenceClassification
import pytorch_lightning as pl

#日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

C:\Users\kinar\.conda\envs\natural_langugage_processing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,num_labels=2
)
bert_sc = bert_sc.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]
label_list = [1,0,1]

#データの符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items()}
labels = torch.tensor(label_list).cuda()

#推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits#分類スコア
labels_predicted = scores.argmax(-1)#スコアが最も値ラベル
num_correct = (labels_predicted==labels).sum().item()#正解数
accuracy = num_correct/labels.size(0)#精度

print("# scoresのサイズ:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)

# scoresのサイズ:
torch.Size([3, 2])
# predicted labels:
tensor([0, 0, 0], device='cuda:0')
# accuracy:
0.3333333333333333


In [15]:
#符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding['labels']=torch.tensor(label_list)#入力にラベルを加える
encoding = {k: v.cuda() for k, v in encoding.items()}

#ロスの計算
output = bert_sc(**encoding)
loss = output.loss#損失の取得
print(loss)

tensor(0.7363, device='cuda:0', grad_fn=<NllLossBackward0>)


In [16]:
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz

'wget' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。
tar: Error opening archive: Failed to open 'ldcc-20140209.tar.gz'


In [18]:
#データローダの作成
dataset_for_loader = [
    #data:2次元テンソルのデータ　labels：データのクラス
    {'data':torch.tensor([0,1]),'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]),'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]),'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]),'labels':torch.tensor(3)},
]
#リストで4つあるデータを2つのデータ(2バッチ)で分割
loader = DataLoader(dataset_for_loader,batch_size=2)

#データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)
    ##ファインチューニングではここでミニバッチごとの処理を行う

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [20]:
#データをシャッフルしてミニバッチ単位に分割
loader = DataLoader(dataset_for_loader,batch_size=2,shuffle=True)

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

# batch 0
{'data': tensor([[6, 7],
        [0, 1]]), 'labels': tensor([3, 0])}
# batch 1
{'data': tensor([[4, 5],
        [2, 3]]), 'labels': tensor([2, 1])}


In [22]:
#全記事の文章データを取得して前処理

#カテゴリーのリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

#トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

#各データの形式を整える
max_length = 128
dataset_for_loader = []
for label,category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./text/{category}/{category}*'):
        lines = open(file,encoding='utf-8').read().splitlines()
        text = '\n' .join(lines[3:])#ファイルの4行目から抜き出す
        encoding = tokenizer(
            text,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )
        encoding['labels'] = label #ラベルを追加
        encoding = {k:torch.tensor(v) for k, v in encoding.items()}
        dataset_for_loader.append(encoding)

100%|██████████| 9/9 [00:13<00:00,  1.53s/it]


In [23]:
print(dataset_for_loader[0])

{'input_ids': tensor([    2,  2340, 19693, 10585, 28459,    35,  6692, 28493,    13,   501,
           62,   101,    37,     8,   569,   335,     5,    51,     7,     9,
         1040,     5,   616,     9,  2941,    18,  5602,   501,    20,    16,
         4027, 10531,   140,    36,    73, 30020, 28457, 25127,    38,  1080,
            5,    53,    28,   707,     5,    12,     9,    80,  3635,   205,
           29,  2935,   604,  5846,  6503,    11,  4722,    16,   861,    13,
            6, 12272, 24050,  2079,    11,    26,    62,    45,    28,  2451,
           80,     8,    36, 24050,    14,    31,  1058,    75, 11218, 10531,
         3676,   542,     5, 22130,     6,  5408,    16,  4831,    80,    29,
           18,  7045,    26, 28456,  4799,   900,     6,   569,   335,     9,
         1704,  1277,    15,  3318,  2575,    29,  2935,  5233,    75,    13,
         3472,   459,    12,  8585,  3171,   312,  3676,   542, 22130,   241,
            5,   709, 28696,  2180,    14, 12959, 